libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import streamlit as st
import matplotlib.pyplot as plt
import tensorflow as tf
import chess.pgn
import random
import chess
from chess import Board, SQUARES, Move
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import stats

c:\Users\garre\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


df

In [2]:
df = pd.read_csv('df.csv')
df

,Game ID,White Result,Black Result,Color,Move Number,Move,Piece
0,1,1,0,White,1,a2a4,Pawn
1,1,1,0,Black,2,g8f6,Knight
2,1,1,0,White,3,d2d4,Pawn
3,1,1,0,Black,4,d7d5,Pawn
4,1,1,0,White,5,g1f3,Knight
...,...,...,...,...,...,...,...
2529402,29319,1,0,White,139,a7d4,Queen
2529403,29319,1,0,Black,140,f6f7,King
2529404,29319,1,0,White,141,a6a7,Pawn
2529405,29319,1,0,Black,142,c6c5,Pawn


cache

In [3]:
import pickle

# Load the cache from the pickle file
with open("full_cache.pkl", "rb") as file:
    cache = pickle.load(file)

In [4]:
cache

{'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1': 'd2d4',
 'rnbqkbnr/pppppppp/8/8/P7/8/1PPPPPPP/RNBQKBNR b KQkq - 0 1': 'e7e5',
 'rnbqkb1r/pppppppp/5n2/8/P7/8/1PPPPPPP/RNBQKBNR w KQkq - 1 2': 'd2d4',
 'rnbqkb1r/pppppppp/5n2/8/P2P4/8/1PP1PPPP/RNBQKBNR b KQkq - 0 2': 'e7e6',
 'rnbqkb1r/ppp1pppp/5n2/3p4/P2P4/8/1PP1PPPP/RNBQKBNR w KQkq - 0 3': 'g1f3',
 'rnbqkb1r/ppp1pppp/5n2/3p4/P2P4/5N2/1PP1PPPP/RNBQKB1R b KQkq - 1 3': 'c8f5',
 'rn1qkb1r/ppp1pppp/5n2/3p1b2/P2P4/5N2/1PP1PPPP/RNBQKB1R w KQkq - 2 4': 'c1f4',
 'rn1qkb1r/ppp1pppp/5n2/3p1b2/P2P3N/8/1PP1PPPP/RNBQKB1R b KQkq - 3 4': 'f5e4',
 'rn1qkb1r/ppp1pppp/5n2/3p4/P2Pb2N/8/1PP1PPPP/RNBQKB1R w KQkq - 4 5': 'e2e3',
 'rn1qkb1r/ppp1pppp/5n2/3p4/P2Pb2N/5P2/1PP1P1PP/RNBQKB1R b KQkq - 0 5': 'e4g6',
 'rn1qkb1r/ppp1pppp/5nb1/3p4/P2P3N/5P2/1PP1P1PP/RNBQKB1R w KQkq - 1 6': 'b1c3',
 'rn1qkb1r/ppp1pppp/5nb1/3p4/P2P3N/2N2P2/1PP1P1PP/R1BQKB1R b KQkq - 2 6': 'e7e6',
 'rn1qkb1r/pp2pppp/5nb1/2pp4/P2P3N/2N2P2/1PP1P1PP/R1BQKB1R w KQkq - 0 7': 'e2e4',


test models accuracy for early game / mid game / late game

first 10 moves

In [38]:
first_10_moves = {}

for key in cache:
    move_number = int(key.split()[-1])
    if 1 <= move_number <= 10:
        first_10_moves[key] = cache[key]

In [39]:
first_10_moves

{'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1': 'd2d4',
 'rnbqkbnr/pppppppp/8/8/P7/8/1PPPPPPP/RNBQKBNR b KQkq - 0 1': 'e7e5',
 'rnbqkb1r/pppppppp/5n2/8/P7/8/1PPPPPPP/RNBQKBNR w KQkq - 1 2': 'd2d4',
 'rnbqkb1r/pppppppp/5n2/8/P2P4/8/1PP1PPPP/RNBQKBNR b KQkq - 0 2': 'e7e6',
 'rnbqkb1r/ppp1pppp/5n2/3p4/P2P4/8/1PP1PPPP/RNBQKBNR w KQkq - 0 3': 'g1f3',
 'rnbqkb1r/ppp1pppp/5n2/3p4/P2P4/5N2/1PP1PPPP/RNBQKB1R b KQkq - 1 3': 'c8f5',
 'rn1qkb1r/ppp1pppp/5n2/3p1b2/P2P4/5N2/1PP1PPPP/RNBQKB1R w KQkq - 2 4': 'c1f4',
 'rn1qkb1r/ppp1pppp/5n2/3p1b2/P2P3N/8/1PP1PPPP/RNBQKB1R b KQkq - 3 4': 'f5e4',
 'rn1qkb1r/ppp1pppp/5n2/3p4/P2Pb2N/8/1PP1PPPP/RNBQKB1R w KQkq - 4 5': 'e2e3',
 'rn1qkb1r/ppp1pppp/5n2/3p4/P2Pb2N/5P2/1PP1P1PP/RNBQKB1R b KQkq - 0 5': 'e4g6',
 'rn1qkb1r/ppp1pppp/5nb1/3p4/P2P3N/5P2/1PP1P1PP/RNBQKB1R w KQkq - 1 6': 'b1c3',
 'rn1qkb1r/ppp1pppp/5nb1/3p4/P2P3N/2N2P2/1PP1P1PP/R1BQKB1R b KQkq - 2 6': 'e7e6',
 'rn1qkb1r/pp2pppp/5nb1/2pp4/P2P3N/2N2P2/1PP1P1PP/R1BQKB1R w KQkq - 0 7': 'e2e4',


In [40]:
len(first_10_moves)

149534

first 20 moves

In [41]:
first_20_moves = {}

for key in cache:
    move_number = int(key.split()[-1])
    if 1 <= move_number <= 20:
        first_20_moves[key] = cache[key]

In [42]:
len(first_20_moves)

650326

first 30 moves

In [43]:
first_30_moves = {}

for key in cache:
    move_number = int(key.split()[-1])
    if 1 <= move_number <= 30:
        first_30_moves[key] = cache[key]

In [44]:
len(first_30_moves)

1136968

modeling

In [45]:
legal_moves = []

for a in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']:
    for b in ['1', '2', '3', '4', '5', '6', '7', '8']:
        for c in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']:
            for d in ['1', '2', '3', '4', '5', '6', '7', '8']:
                legal_moves.append(a + b + c + d)
                if d == '1' or d == '8':
                    legal_moves.append(a + b + c + d + 'r')
                    legal_moves.append(a + b + c + d + 'q')
                    legal_moves.append(a + b + c + d + 'n')
                    legal_moves.append(a + b + c + d + 'b')

In [46]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
label_encoder = LabelEncoder()

label_encoder.fit(legal_moves)

LabelEncoder()

In [47]:
# Define a function to convert a board object to a one-hot encoded representation
def board_to_tensor(board):
    piece_mapping = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }
    tensor = np.zeros((8, 8, 12), dtype=np.float32)
    for square in chess.SQUARES:
        piece_type = board.piece_type_at(square)
        if piece_type is not None:
            piece_index = piece_mapping[chess.piece_symbol(piece_type)]
            row = square // 8
            col = square % 8
            tensor[row, col, piece_index] = 1
    return tensor

In [48]:
# Iterate over your cache dictionary and convert board states to tensors
first_10_tensor = {}
for fen, move in first_10_moves.items():
    board = Board(fen)
    tensor = board_to_tensor(board)
    first_10_tensor[fen] = tensor

# Convert first_10_tensor to a TensorFlow tensor if desired
first_10_tensor = tf.convert_to_tensor(list(first_10_tensor.values()))

first_10_array = np.array(first_10_tensor)

In [50]:
best_moves_10 = list(first_10_moves.values())

In [52]:
# Fit and transform the labels in best_moves
encoded_best_moves_10 = label_encoder.transform(best_moves_10)

In [53]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train10, X_test10, y_train10, y_test10 = train_test_split(first_10_array, encoded_best_moves_10, test_size=0.2, random_state=42)

In [55]:
# Iterate over your cache dictionary and convert board states to tensors
first_20_tensor = {}
for fen, move in first_20_moves.items():
    board = Board(fen)
    tensor = board_to_tensor(board)
    first_20_tensor[fen] = tensor

# Convert first_20_tensor to a TensorFlow tensor if desired
first_20_tensor = tf.convert_to_tensor(list(first_20_tensor.values()))

first_20_array = np.array(first_20_tensor)

In [57]:
best_moves_20 = list(first_20_moves.values())

In [58]:
# Fit and transform the labels in best_moves
encoded_best_moves_20 = label_encoder.transform(best_moves_20)

In [59]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train20, X_test20, y_train20, y_test20 = train_test_split(first_20_array, encoded_best_moves_20, test_size=0.2, random_state=42)

In [61]:
# Iterate over your cache dictionary and convert board states to tensors
first_30_tensor = {}
for fen, move in first_30_moves.items():
    board = Board(fen)
    tensor = board_to_tensor(board)
    first_30_tensor[fen] = tensor

# Convert first_30_tensor to a TensorFlow tensor if desired
first_30_tensor = tf.convert_to_tensor(list(first_30_tensor.values()))

first_30_array = np.array(first_30_tensor)

In [63]:
best_moves_30 = list(first_30_moves.values())

In [64]:
# Fit and transform the labels in best_moves
encoded_best_moves_30 = label_encoder.transform(best_moves_30)

In [65]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train30, X_test30, y_train30, y_test30 = train_test_split(first_30_array, encoded_best_moves_30, test_size=0.2, random_state=42)

3 models representing first 10 moves of each game, first 20 moves of each game, and first 30 moves of each game

load in models

In [71]:
model20 = tf.keras.models.load_model('first_20_model.h5')
model40 = tf.keras.models.load_model('first_40_model.h5')
model60 = tf.keras.models.load_model('first_60_model.h5')

metrics for each model

In [72]:
metrics10 = model10.evaluate(X_test10, y_test10)

935/935 [==============================] - 4s 4ms/step - loss: 3.4154 - accuracy: 0.2776


In [73]:
metrics20 = model20.evaluate(X_test20, y_test20)

4065/4065 [==============================] - 15s 4ms/step - loss: 4.5891 - accuracy: 0.1224


In [74]:
metrics30 = model30.evaluate(X_test30, y_test30)

7107/7107 [==============================] - 25s 4ms/step - loss: 4.9476 - accuracy: 0.0751
